In [1]:
import numpy as np
from hmm_utils import HMM
from attackers.smoothing_state_attacker import ss_attacker
from attackers.decoding_attacker import dec_attacker
from params import *
from solvers.mc_ennumeration import MC_enumeration
#from solvers.simulated_annealing import simulated_annealing
#from solvers.aps_gibbs import aps_gibbs
from solvers.nn_RS.nn_RS import *
#from solvers.aps_gibbs_class import *

from solvers.aps_gibbs_class_exp_decay import *
from solvers.random_mc_ennumeration import *

from scipy.special import softmax

def ohe(x, n_obs):
    return np.diag(np.ones(n_obs))[ x.astype(int) ]

In [2]:
data = np.load("data/HMM_structure_experiment_comb3_X_30_T_10_Q_10.npy", allow_pickle=True).tolist()
priors     = data["prior"]
transition = data["trans"]
emission   = data["emiss"]
rho_probs = 1.0*np.ones(30)
x_vector = data["X"]

In [3]:
hmm = HMM(n_components=10, n_obs=30 )
hmm.emissionprob_ = emission
hmm.transmat_     = transition
hmm.startprob_    = priors


In [4]:
V, seq = hmm.nu(x_vector)
seq

array([4, 5, 4, 2, 6, 7, 4, 0, 1, 2])

In [5]:
attacker = dec_attacker(priors, transition, emission, rho_probs,
         x_vector, w1 = 10.0 ,w2 = 0.0 , seq = 0*np.ones(10, dtype='int'), k_value=10000)

In [7]:
N = 10
t = 5

# Random MC

In [ ]:
rmc = random_mc_ennumeration(attacker)

### t = 5s

In [12]:
qualities = np.zeros(N)
for i in range(N):
    print(i)
    z_star, quality = rmc.iterate(simulation_seconds=5)
    qualities[i] = quality

0


/home/roi/MEGA/posdoc_icmat/projects/poisoning_hmms/hmm_utils.py:51: RuntimeWarning: divide by zero encountered in log
  lemissionprob_ = np.log(self.emissionprob_)


1
2
3
4
5
6
7
8
9


In [13]:
print("Mean Expected Utility", np.mean(qualities) )
print("Std Expected Utility", np.std(qualities) )

Mean Expected Utility 123.09743464463108
Std Expected Utility 1.698970681094775


### t = 60s

In [14]:
qualities = np.zeros(N)
for i in range(N):
    print(i)
    z_star, quality = rmc.iterate(simulation_seconds=60)
    qualities[i] = quality

0
1
2
3
4
5
6
7
8
9


In [15]:
print("Mean Expected Utility", np.mean(qualities) )
print("Std Expected Utility", np.std(qualities) )

Mean Expected Utility 127.51966633731547
Std Expected Utility 2.18838051637791


# APS

In [19]:
from solvers.aps_gibbs_class import aps_gibbs
aps = aps_gibbs(attacker, np.arange(500, 10000, 10))

### t = 5s

In [20]:
qualities = np.zeros(N)
for i in range(N):
    print(i)
    z_star, quality = aps.iterate(simulation_seconds=5)
    qualities[i] = quality

0
1
2
3
4
5
6
7
8
9


In [21]:
print("Mean Expected Utility", np.mean(qualities) )
print("Std Expected Utility", np.std(qualities) )

Mean Expected Utility 158.2838559386976
Std Expected Utility 0.47801000283714906


### t = 60s

In [22]:
qualities = np.zeros(N)
for i in range(N):
    print(i)
    z_star, quality = aps.iterate(simulation_seconds=60)
    qualities[i] = quality

0
1
2
3
4
5
6
7
8
9


In [23]:
print("Mean Expected Utility", np.mean(qualities) )
print("Std Expected Utility", np.std(qualities) )

Mean Expected Utility 157.56709603145208
Std Expected Utility 1.789653854139626


In [25]:
attack_obs = attacker.attack_X(np.ones_like(z_star), z_star)
V, seq = hmm.nu(attack_obs.astype(int))
seq

array([0, 0, 0, 8, 0, 0, 8, 0, 0, 0])